In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

Setting environment for Gordon


In [2]:
from scipy.interpolate import RectBivariateSpline
from skimage.transform import resize

In [3]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']

structures = paired_structures + singular_structures

In [18]:
for stack in ['MD585']:
        
    sections_to_filenames = metadata_cache['sections_to_filenames'][stack]
    first_sec, last_sec = metadata_cache['section_limits'][stack]
    anchor_fn = metadata_cache['anchor_fn'][stack]
    
    train_sample_scheme = 1
    
    bar = show_progress_bar(first_sec, last_sec)
    
#     for sec in range(84, 84+1):
    for sec in range(200, 201):
#     for sec in range(first_sec, last_sec+1):

        bar.value = sec
        
        print sec

        fn = sections_to_filenames[sec]
        if fn in ['Nonexisting', 'Rescan', 'Placeholder']:
            continue

        # output
        scoremaps_dir = os.path.join(SCOREMAPS_ROOTDIR, stack, 
                                     '%(fn)s_lossless_alignedTo_%(anchor_fn)s_cropped' % \
                                     dict(stack=stack, fn=fn, anchor_fn=anchor_fn))
        create_if_not_exists(scoremaps_dir)

        ## define grid, generate patches

        t = time.time()

        locations_fn = PATCH_FEATURES_ROOTDIR + '/%(stack)s/%(fn)s_lossless_alignedTo_%(anchor_fn)s_cropped/%(fn)s_lossless_alignedTo_%(anchor_fn)s_cropped_patch_locations.txt' % dict(stack=stack, fn=fn, anchor_fn=anchor_fn)

        with open(locations_fn, 'r') as f:
            sample_locations_roi = np.array([map(int, line.split()[1:]) for line in f.readlines()])
        
        ## interpolate
        
        interpolation_xmin, interpolation_ymin = sample_locations_roi.min(axis=0)
        interpolation_xmax, interpolation_ymax = sample_locations_roi.max(axis=0)
        interpolation_w = interpolation_xmax - interpolation_xmin + 1
        interpolation_h = interpolation_ymax - interpolation_ymin + 1

        ##### sample_locations_roi + scores to dense_score_map #####

        shrink_factor = 4 # do interpolation on a smaller grid, then resize to original dimension

        sample_locations_unique_xs = np.unique(sample_locations_roi[:,0])
        sample_locations_unique_ys = np.unique(sample_locations_roi[:,1])

        n_sample_x = sample_locations_unique_xs.size
        n_sample_y = sample_locations_unique_ys.size

        index_x = dict([(j,i) for i,j in enumerate(sample_locations_unique_xs)])
        index_y = dict([(j,i) for i,j in enumerate(sample_locations_unique_ys)])
        sample_location_indices = np.asarray([(index_x[x], index_y[y]) for x, y in sample_locations_roi])

        sample_locations_interpolatedArea_ys_matrix, \
        sample_locations_interpolatedArea_xs_matrix = np.meshgrid(range(interpolation_ymin/shrink_factor, 
                                                                        interpolation_ymax/shrink_factor), 
                                                                  range(interpolation_xmin/shrink_factor, 
                                                                        interpolation_xmax/shrink_factor), 
                                                                  indexing='ij')

#         sparse_score_dir = create_if_not_exists(os.path.join(SPARSE_SCORES_ROOTDIR, stack, '%(fn)s_lossless_alignedTo_%(anchor_fn)s_cropped' % \
#                                       {'fn': fn, 'anchor_fn': anchor_fn}))

#         probs_allClasses = {label: bp.unpack_ndarray_file(sparse_score_dir + '/%(fn)s_lossless_alignedTo_%(anchor_fn)s_cropped_%(label)s_sparseScores.hdf' % \
#                     {'fn': fn, 'anchor_fn': anchor_fn, 'label':label})
#                             for label in structures}

        probs_allClasses = {label: DataManager.load_sparse_scores(stack, fn=fn, anchor_fn=anchor_fn, 
                                                  label=label, train_sample_scheme=train_sample_scheme)
                            for label in structures}

        sys.stderr.write('preprocess: %.2f seconds\n' % (time.time() - t))
        
        def generate_score_map(label):

            if label == 'BackG':
                return None
            
            score_matrix = np.zeros((n_sample_x, n_sample_y))
            score_matrix[sample_location_indices[:,0], sample_location_indices[:,1]] = probs_allClasses[label]

            spline = RectBivariateSpline(sample_locations_unique_xs/shrink_factor, 
                                         sample_locations_unique_ys/shrink_factor, 
                                         score_matrix, 
                                         bbox=[interpolation_xmin/shrink_factor, 
                                               interpolation_xmax/shrink_factor, 
                                               interpolation_ymin/shrink_factor, 
                                               interpolation_ymax/shrink_factor])

#             t = time.time()
            dense_score_map = spline.ev(sample_locations_interpolatedArea_xs_matrix, 
                                        sample_locations_interpolatedArea_ys_matrix)
#             sys.stderr.write('evaluate spline: %.2f seconds\n' % (time.time() - t))
            
            t1 = time.time()
            dense_score_map = resize(dense_score_map, (interpolation_h, interpolation_w)) # similar speed as rescale
#             dense_score_map = rescale(dense_score_map, shrink_factor)
            sys.stderr.write('scale up: %.2f seconds\n' % (time.time() - t1))

#             t = time.time()
            dense_score_map[dense_score_map < 1e-1] = 0
            dense_score_map[dense_score_map > 1.] = 1.
#             sys.stderr.write('threshold: %.2f seconds\n' % (time.time() - t))

            if np.count_nonzero(dense_score_map) < 1e5:
                sys.stderr.write('No %s is detected on section %d\n' % (label, sec))
                return None
            
            scoremap_bp_filepath, scoremap_interpBox_filepath = DataManager.get_scoremap_filepath(stack=stack, fn=fn, anchor_fn=anchor_fn, label=label,
                                                                    return_bbox_fp=True, train_sample_scheme=train_sample_scheme)

            save_hdf(dense_score_map.astype(np.float16), scoremap_bp_filepath, complevel=5)
            
            np.savetxt(scoremap_interpBox_filepath,
                   np.array((interpolation_xmin, interpolation_xmax, interpolation_ymin, interpolation_ymax))[None],
                   fmt='%d')
            
        
        t = time.time()
        
        # if too many disk saves are simultaneous, they will be conflicting, so split into two sessions
#         for i in range(0, len(structures), 8):
#             _ = Parallel(n_jobs=16)(delayed(generate_score_map)(l) for l in structures[i:i+15])

        _ = Parallel(n_jobs=15)(delayed(generate_score_map)(l) for l in structures)
 
        sys.stderr.write('interpolate: %.2f seconds\n' % (time.time() - t)) # ~ 30 seconds / section on one node

200


preprocess: 1.95 seconds
scale up: 9.82 seconds
scale up: 9.86 seconds
scale up: 9.86 seconds
scale up: 9.88 seconds
scale up: 9.93 seconds
scale up: 9.94 seconds
scale up: 9.95 seconds
scale up: 9.96 seconds
scale up: 9.98 seconds
scale up: 9.97 seconds
scale up: 10.00 seconds
scale up: 10.03 seconds
scale up: 10.03 seconds
scale up: 10.03 seconds
scale up: 10.05 seconds
scale up: 9.42 seconds
scale up: 9.54 seconds
scale up: 9.52 seconds
scale up: 9.67 seconds
scale up: 9.60 seconds
scale up: 9.58 seconds
scale up: 9.67 seconds
scale up: 9.72 seconds
scale up: 9.71 seconds
scale up: 9.68 seconds
scale up: 9.63 seconds
scale up: 9.72 seconds
scale up: 9.70 seconds
interpolate: 38.86 seconds
